In [1]:
import pandas as pd
from collections import Counter

In [2]:
df_game = pd.read_csv('csvs/game.csv')[['match_id','game_id','game_nbr','score_w','score_l','w_team_id','l_team_id','skill_lvl']]
df_game['game_id'] = [int(x[1:]) for x in df_game.game_id]
df_game['match_id'] = [int(x[1:]) for x in df_game.match_id]
df_game = df_game.sort_values(['match_id','game_id']).reset_index(drop=True)
print('GAME:\n ')
df_game.head(2)

GAME:
 


,match_id,game_id,game_nbr,score_w,score_l,w_team_id,l_team_id,skill_lvl
0,1,1,1,12,10,T2,T1,Pro
1,1,2,2,11,7,T1,T2,Pro


In [3]:
df_rally = pd.read_csv('csvs/rally.csv')
df_rally = df_rally[[
    'match_id', 'game_id', 'rally_id', 'point_nbr', 'srv_team_id',
    'rtrn_team_id', 'w_team_id', 'to_ind', 'to_team_id', 'ending_type'
]]
df_rally['match_id'] = [int(x[1:]) for x in df_rally.match_id]
df_rally['game_id'] = [int(x[1:]) for x in df_rally.game_id]
df_rally['rally_id'] = [int(x[1:]) for x in df_rally.rally_id]
df_rally = df_rally.sort_values(
    ['match_id', 'game_id', 'rally_id', 'point_nbr']).reset_index(drop=True)
print('RALLY:\n')
df_rally.head(2)

RALLY:



,match_id,game_id,rally_id,point_nbr,srv_team_id,rtrn_team_id,w_team_id,to_ind,to_team_id,ending_type
0,1,1,1,1,T1,T2,T2,N,NaN,Other
1,1,1,2,2,T2,T1,T2,N,NaN,Winner


In [4]:
df_numrallys = df_rally[['game_id','point_nbr']].groupby(['game_id'],axis=0).max()
print(f"min # rallies: {int(df_numrallys.min())}")
print(f"avg # rallies: {float(df_numrallys.mean().round(1))}")
print(f"med # rallies: {int(df_numrallys.median())}")
print(f"max # rallies: {int(df_numrallys.max())}")

min # rallies: 20
avg # rallies: 47.5
med # rallies: 48
max # rallies: 71


In [5]:
numtos = []
numrallys = []
for i in range(1, 1 + len(df_game)):
    one_game = df_rally[df_rally.game_id == i]
    todict = list(Counter(one_game.to_ind).items())
    if len(todict) == 1:
        numtos.append(0)
    else:
        numtos.append(todict[1][1])
    numrallys.append(int(one_game.point_nbr.max()))

In [6]:
df_metrx = pd.DataFrame({'game_id':list(range(1,1+len(df_game.game_id))),'timeouts':numtos,'rallys':numrallys})
df_metrx['avg_rallies_noto'] = round(df_metrx.rallys / (1 + df_metrx.timeouts), 1)
print('per game -> total # timeouts taken, total # rallies played, and avg # of rallies between timeouts:')
print('(showing 5 at random)')
df_metrx

per game -> total # timeouts taken, total # rallies played, and avg # of rallies between timeouts:
(showing 5 at random)


,game_id,timeouts,rallys,avg_rallies_noto
0,1,4,63,12.6
1,2,3,54,13.5
2,3,1,38,19.0
3,4,3,60,15.0
4,5,1,37,18.5
5,6,2,43,14.3
6,7,2,41,13.7
7,8,1,53,26.5
8,9,2,54,18.0
9,10,1,33,16.5


In [7]:
print('per number of total timeouts taken in a game -> avg of the average # of rallies between timeouts:')
df_avgunbrk = df_metrx[['timeouts','avg_rallies_noto']].groupby('timeouts').mean().round(1).reset_index(drop=False)
df_avgunbrk.columns = [['timeouts','avg^2_rallies_noto']]
df_avgunbrk

per number of total timeouts taken in a game -> avg of the average # of rallies between timeouts:


,timeouts,avg^2_rallies_noto
0,0,36.9
1,1,23.9
2,2,15.8
3,3,14.6
4,4,12.6


In [8]:
print('to illustrate approach, choose longest game with exactly 2 timeouts taken:')
exgameid=list(df_metrx[df_metrx.timeouts == 2].sort_values('avg_rallies_noto',ascending=False)['game_id'])[0]
print(f'\t-game_id = #{exgameid}')

to illustrate approach, choose longest game with exactly 2 timeouts taken:
	-game_id = #23


In [9]:
print('check to see if both teams take a timeout for our chosen example game:')
df_exgame = df_rally[df_rally.game_id==exgameid]
if len(set(df_exgame[df_exgame.to_ind == 'Y'].to_team_id)) == 2:
    print('\t-Success')

check to see if both teams take a timeout for our chosen example game:
	-Success


In [10]:
print(df_exgame.reset_index(drop=True).to_string(index=False))

 match_id  game_id  rally_id  point_nbr srv_team_id rtrn_team_id w_team_id to_ind to_team_id    ending_type
       14       23      1064          1         T20          T21       T20      N        NaN Unforced Error
       14       23      1065          2         T20          T21       T21      N        NaN         Winner
       14       23      1066          3         T21          T20       T20      N        NaN          Other
       14       23      1067          4         T21          T20       T20      N        NaN Unforced Error
       14       23      1068          5         T20          T21       T21      N        NaN Unforced Error
       14       23      1069          6         T20          T21       T21      N        NaN         Winner
       14       23      1070          7         T21          T20       T21      N        NaN Unforced Error
       14       23      1071          8         T21          T20       T21      N        NaN Unforced Error
       14       23      1072

In [11]:
df_exgame.to_csv('game23.csv')

In [12]:
print('previous timeout was at end of 20 min Tuesday Night Pickleball, lets try 4 timeouts:')
exgameid=list(df_metrx[df_metrx.timeouts == 4].sort_values('avg_rallies_noto',ascending=False)['game_id'])[0]
print(f'\t-game_id = #{exgameid}')

previous timeout was at end of 20 min Tuesday Night Pickleball, lets try 4 timeouts:
	-game_id = #1


In [13]:
print('check to see if both teams take a timeout for our chosen example game:')
df_exgame = df_rally[df_rally.game_id==exgameid]
if len(set(df_exgame[df_exgame.to_ind == 'Y'].to_team_id)) == 2:
    print('\t-Success')
else:
    print('\t-Fail')

check to see if both teams take a timeout for our chosen example game:
	-Success


In [14]:
print(df_exgame.reset_index(drop=True).to_string(index=False))

 match_id  game_id  rally_id  point_nbr srv_team_id rtrn_team_id w_team_id to_ind to_team_id    ending_type
        1        1         1          1          T1           T2        T2      N        NaN          Other
        1        1         2          2          T2           T1        T2      N        NaN         Winner
        1        1         3          3          T2           T1        T2      N        NaN Unforced Error
        1        1         4          4          T2           T1        T2      N        NaN Unforced Error
        1        1         5          5          T2           T1        T1      N        NaN          Other
        1        1         6          6          T2           T1        T1      N        NaN          Other
        1        1         7          7          T1           T2        T2      N        NaN          Other
        1        1         8          8          T1           T2        T2      N        NaN Unforced Error
        1        1         9

In [15]:
df_exgame.to_csv('game1.csv')

In [16]:
print('straight to game2:')
exgameid=2
print(f'\t-game_id = #{exgameid}')

straight to game2:
	-game_id = #2


In [17]:
print('check to see if both teams take a timeout for our chosen example game:')
df_exgame = df_rally[df_rally.game_id==exgameid]
if len(set(df_exgame[df_exgame.to_ind == 'Y'].to_team_id)) == 2:
    print('\t-Success')
else:
    print('\t-Fail')

check to see if both teams take a timeout for our chosen example game:
	-Success


In [18]:
df_exgame.to_csv(f'game{exgameid}.csv')